Lets create a Petroleum engineering assistant that uses custom tools to complete petroleum engineering calculations: we will  build a model that can calculate STOIP: Stock Tank Oil In Place 

First, lets sort out imports

In [1]:
from openai import OpenAI
import gradio as gr
import json

lets setup our tools, a function that computes Stock Tank Oil in Place

In [2]:
def compute_stoip(area,thickness,water_saturation,oil_formation_volume_factor):
    return ( area * thickness * (1 - water_saturation) )/oil_formation_volume_factor

stoip_tool = {
    "type":"function",
    "function":{
        "name":"compute_stoip",
        "description":"Compute Stock Tank Oil in Place",
        "parameters":{
            "type":"object",
            "properties":{"area":{"type":"number"},"thickness":{"type":"number"},"water_saturation":{"type":"number"},"oil_formation_volume_factor":{"type":"number"}}
        }
    }
}

Lets test our tool

In [3]:
compute_stoip(100,10,0.2,1.1)

727.2727272727273

Now, lets handle message where tool is needed

In [4]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        print(tool_call)
        if(tool_call.function.name == "compute_stoip"):
            arguments = json.loads(tool_call.function.arguments)
            area = arguments["area"]
            thickness = arguments["thickness"]
            water_saturation = arguments["water_saturation"] 
            oil_formation_volume_factor = arguments["oil_formation_volume_factor"]
            stoip = compute_stoip(area,thickness,water_saturation,oil_formation_volume_factor)
            responses.append({
                "role": "tool",
                "content": str(stoip),
                "tool_call_id": tool_call.id
            })
    return responses

Great, tool is fine, now lets create a chat function that can work with Gradio

In [7]:
system_prompt = "You are a helpful Petroleum Engineer assistant that can help with calculations. You have access to a tool that can compute Stock Tank Oil in Place. You can use this tool to help the user with their calculations."

In [5]:
def chat(message,history):
    history = [{"role":h["role"],"content":h["content"]} for h in history]
    messages = [{"role":"system","content":system_prompt}] + history + [{"role":"user","content":message}]
    openai = OpenAI()
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=[stoip_tool]
    )
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message.tool_calls)
        tool_responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(tool_responses)
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )

    return response.choices[0].message.content

Now, lets setup gradio

In [8]:
gr.ChatInterface(chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7892
* To create a public link, set `share=True` in `launch()`.


[ChatCompletionMessageFunctionToolCall(id='call_l7jCQSikuhbV4b48tFxrKw22', function=Function(arguments='{"area":2500,"thickness":65,"water_saturation":0.28,"oil_formation_volume_factor":1.18}', name='compute_stoip'), type='function')]
ChatCompletionMessageFunctionToolCall(id='call_l7jCQSikuhbV4b48tFxrKw22', function=Function(arguments='{"area":2500,"thickness":65,"water_saturation":0.28,"oil_formation_volume_factor":1.18}', name='compute_stoip'), type='function')
